# MYMail_review – Notas / comandos rápidos

Este notebook sirve como recordatorio de comandos para:
- Subir nuevos ficheros a **entrada** (CosmosDB)
- Gestión de **usuarios** (CosmosDB)
- Despliegue de imagen (ACR) + restart App Service


## 0) CosmosDB (setup + migración)

Variables mínimas en `.env` / App Service Configuration:
- (Cosmos-only) no se usa `STORAGE_BACKEND`.
- `COSMOS_ENDPOINT=<URI>`
- `COSMOS_KEY=<KEY>`
- `COSMOS_DATABASE=mymailreview`

Crear DB/containers (una vez):


Test de conexión a CosmosDB:


In [ ]:
!python Testing/test_cosmos.py


## 1) Subir un nuevo Excel a ENTRADA (CosmosDB)

Carga el Excel y crea registros en el contenedor `entrada` de CosmosDB.

Nota: si un `record_json` supera el límite seguro, se guarda el payload en Blob (legacy) y en Cosmos queda una referencia.


In [ ]:
# Ya no se usa Azure Blob/Table Storage. La ingesta de entrada ya está migrada a CosmosDB.


In [ ]:
# Si necesitas reingestar, crea un script Cosmos-first o reutiliza tu pipeline de carga.


In [ ]:
# (Deprecated) load_entrada.py eliminado al pasar a Cosmos-only.


Flags útiles:
- `--replace` borra la partición antes de cargar
- `--partition` PartitionKey destino (por defecto: `default`)


## 2) Gestión de usuarios (CosmosDB)

### 2.1 Crear/actualizar usuario


In [ ]:
!python scripts/manage_users.py add --username "usuario1" --password "MiPassword" --role revisor

In [ ]:
!python scripts/manage_users.py add --username "admin" --password "MiPassword" --role admin

### 2.2 Cambiar rol


In [ ]:
!python scripts/manage_users.py set-role --username "usuario1" --role admin

### 2.3 Actualizar contraseña


In [ ]:
!python scripts/manage_users.py set-password --username "usuario1" --password "NuevaPassword"

### 2.4 Listar usuarios


In [ ]:
!python scripts/manage_users.py list

## Requisitos

- Variables en `.env` / `config.py` (CosmosDB: `COSMOS_ENDPOINT`, `COSMOS_KEY`).
- CosmosDB: `COSMOS_ENDPOINT`, `COSMOS_KEY`.
- Dependencias instaladas: `pip install -r requirements.txt`


## 3) Test de Azure OpenAI (legacy)

Usa el script `Testing/test_azure_openai.py` para validar que el endpoint + deployment + api-version funcionan.


In [ ]:
! .\\.venv\\Scripts\\python.exe Testing/test_azure_openai.py --prompt "Responde exactamente: OK" --temperature 1 --max-tokens 256

## 4) Desplegar en cloud (ACR)

Comandos para construir, etiquetar y publicar la imagen en Azure Container Registry.

Antes de empezar:
- Asegúrate de que **Docker Desktop** está iniciado (si no, `docker build` fallará).
- Asegúrate de estar logueado en **Azure**.

Configurar App Service con CosmosDB (placeholders):


In [ ]:
!az webapp config appsettings set --resource-group <RG> --name <APP_NAME> --settings COSMOS_ENDPOINT=<URI> COSMOS_KEY=<KEY> COSMOS_DATABASE=mymailreview


In [ ]:
!docker info


In [ ]:
!az login


In [ ]:
!az acr login -n mymailrevieweuweconreg002


In [7]:
!docker build -t mymailreview .
!docker tag mymailreview mymailrevieweuweconreg002.azurecr.io/mymailreview:latest
!docker push mymailrevieweuweconreg002.azurecr.io/mymailreview:latest


#0 building with "desktop-linux" instance using docker driver

#1 [internal] load build definition from Dockerfile
#1 transferring dockerfile: 374B 0.1s done
#1 DONE 0.1s

#2 [internal] load metadata for docker.io/library/python:3.11-slim
#2 DONE 2.5s

#3 [internal] load .dockerignore
#3 transferring context: 163B 0.0s done
#3 DONE 0.1s

#4 [1/6] FROM docker.io/library/python:3.11-slim@sha256:158caf0e080e2cd74ef2879ed3c4e697792ee65251c8208b7afb56683c32ea6c
#4 DONE 0.0s

#5 [internal] load build context
#5 transferring context: 45.15kB 0.1s done
#5 DONE 0.1s

#6 [2/6] WORKDIR /app
#6 CACHED

#7 [3/6] COPY requirements.txt ./
#7 CACHED

#8 [4/6] RUN pip install --no-cache-dir --upgrade pip     && pip install --no-cache-dir -r requirements.txt
#8 CACHED

#9 [5/6] COPY .env ./.env
#9 CACHED

#10 [6/6] COPY . .
#10 DONE 0.3s

#11 exporting to image
#11 exporting layers 0.1s done
#11 writing image sha256:fe6185b9d827ad03c3e66e39fbda474d7a685b0e952b1cccdf1bf03b7c9dfd35 done
#11 naming to dock

The push refers to repository [mymailrevieweuweconreg002.azurecr.io/mymailreview]
929a03cf2f8d: Preparing
719ed1be02ed: Preparing
851f34a2e2fd: Preparing
6c05da4bcef4: Preparing
a59bbb4f25eb: Preparing
fa384bf02ac1: Preparing
600af8de593b: Preparing
424dc4972605: Preparing
77a2b55fbe8b: Preparing
fa384bf02ac1: Waiting
600af8de593b: Waiting
424dc4972605: Waiting
77a2b55fbe8b: Waiting
6c05da4bcef4: Layer already exists
a59bbb4f25eb: Layer already exists
851f34a2e2fd: Layer already exists
719ed1be02ed: Layer already exists
77a2b55fbe8b: Layer already exists
600af8de593b: Layer already exists
fa384bf02ac1: Layer already exists
424dc4972605: Layer already exists
929a03cf2f8d: Pushed
latest: digest: sha256:8deefbcb86a303ea4c3285097fab19dfb836b15a91bb341e26964b3d0201e235 size: 2202


Reiniciar App Service para que cargue el nuevo contenedor (ajusta `<RG>` y `<APP_NAME>`):


In [ ]:
!az webapp restart --resource-group <RG> --name <APP_NAME>


## OLD (legacy) – Blob / Azure Table Storage

La app antigua usaba Azure **Table Storage** para persistencia y Azure **Blob** para entrada/revisiones.

Variables legacy:
- (Deprecated) Table Storage eliminado.
- `COSMOS_ENDPOINT=...`
- `COSMOS_KEY=...`
- `TABLE_USERS/TABLE_LOGS/TABLE_RESULTADOS/TABLE_DESCARTES/TABLE_ENTRADA`
- `AZURE_BLOB_CONTAINER_ENTRADA=entrada`
- `AZURE_BLOB_CONTAINER_REVISIONES=revisiones`

Notas:
- Aunque uses CosmosDB, el Blob se puede seguir usando para `revisiones` y para payloads grandes de `entrada`.
